## Semantic Kernel Agent Exit Interview

## Prepare file path

In [8]:
import os

# Let's form the file paths that we will later pass to the assistant
file_path = os.path.join("../Data/employeeinfo/","ziggy_zulueta_info.pdf",)

## Create Agent and Thread

In [10]:
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

# Step 1: Create an assistant agent
interview_agent = await AzureAssistantAgent.create(
        kernel=Kernel(),
        service_id="agent",
        name="Interview_Agent",
        instructions="""
            You are an intelligent and empathetic AI agent named Sproutie designed to conduct exit interviews for XYZ Company. 
            Your task is to ask insightful and relevant questions based on the provided resume to understand the employee's experience, reasons for leaving, and any feedback they may have for the company. 
            You will use the information in your file store to tailor your questions and ensure the conversation is personalized and meaningful.
            You will start off by introducing yourself and explaining the purpose of the interview.
            You will then provide one question at a time and wait for the employee's response before proceeding to the next question.
            Once you are done asking all your questions, you provide a closing statement and thank the employee for their time.
        """,
        enable_file_search=True,
        vector_store_filenames=[file_path],
    )

# Step 2: Create a thread
thread_id = await interview_agent.create_thread()

## Helper Function

In [11]:
async def run_agent(user_question):
    # STEP 3: Add a user question to the thread
    await interview_agent.add_chat_message(
            thread_id=thread_id, 
            message=ChatMessageContent(role=AuthorRole.USER, content=user_question)
    )

    # STEP 4: Invoke the agent to get a response
    async for response in interview_agent.invoke(thread_id=thread_id):
        print(f"{response.content}", end="", flush=True)
        

In [13]:
is_complete: bool = False
user_input = "Start the interview. You will use the employee's information to ask relevant questions for the exit interview"
while not is_complete:
    await run_agent(user_input)

    # Ensure 'input' is not shadowed
    if 'input' in globals() and isinstance(globals()['input'], str):
        del globals()['input']
        
    user_input = input("User:> ")
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    print("\n")
    print("User:>", user_input)


Hello Ziggy,

I'm Sproutie, an AI agent designed to conduct exit interviews for XYZ Company. The purpose of this interview is to understand your experience at the company, the reasons for your departure, and any feedback you may have. This will help us improve our work environment and practices.

To start, I would like to ask you a question:

1. Could you tell me more about your overall experience working at XYZ Company, particularly in your role as a Technical Trainer and Architect?

 (Please provide your response and I will proceed with the next question.)

User:> It was fullfilling and challenging
I'm glad to hear that your experience was fulfilling and challenging. Let's delve deeper into some specifics:

2. Can you share key projects or accomplishments during your tenure as a Technical Trainer and Architect that you are particularly proud of?

User:> I liked it that i was able to train a lot of customers and we did a lot of challenging projects
That's wonderful to hear. Engaging i

## Prepare chat history

In [15]:
messages = []
async for message in interview_agent.get_thread_messages(thread_id):
    messages.append(f"{message.role} : {message.content}")
all_messages = "\n".join(messages)

In [16]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory

# Create the instance of the Kernel
kernel = Kernel()

azure_service_id = "azure"
kernel.add_service(AzureChatCompletion(service_id=azure_service_id))

# Create the agent
analyzer_agent = ChatCompletionAgent(
    service_id=azure_service_id, 
    kernel=kernel, 
    name="AnalyzerAgent", 
    instructions="You will be given an interview transcript and you will analyze the interview.", 
)

# Create a chat history
chat = ChatHistory()

input = "Analyze the interview transcript: " + all_messages
# Add the user message to the chat history
chat.add_user_message(input)

async for content in analyzer_agent.invoke(chat):
    # Print the agent response
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

# AuthorRole.ASSISTANT - AnalyzerAgent: 'The provided transcript shows an exit interview with an employee named Ziggy, who worked as a Technical Trainer and Architect at XYZ Company. The assistant (Sproutie) asks a series of structured questions aimed at understanding Ziggy's experiences, challenges, and feedback.

1. **Overall Experience**: 
   - Ziggy describes their experience as fulfilling and challenging, particularly enjoying the opportunity to train customers and work on challenging projects.

2. **Notable Projects or Accomplishments**: 
   - Although not specified in detail, Ziggy’s response hints at pride in the key projects and training they were involved in.

3. **Motivation for Leaving**: 
   - Key reasons for seeking new opportunities included a desire for higher pay and more flexibility in remote work arrangements. The requirement to commute to the office three times per week with a two-hour travel time was notably burdensome.

4. **Areas for Improvement**: 
   - Ziggy id

## Clean up

In [17]:
if interview_agent is not None:
    [await interview_agent.delete_file(file_id) for file_id in interview_agent.file_search_file_ids]
    await interview_agent.delete_thread(thread_id)
    await interview_agent.delete()
